In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(ggplot2)
library(dplyr)
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/output/"
rxn_pca_list <- readRDS(paste(OUT_DIR, "rxn_pca_nls.Rds", sep = ""))
print(length(names(rxn_pca_list)))
vst.count.mtx.train <- readRDS(paste(OUT_DIR, "vst_count_mtx_train.Rds", sep = ""))
rxn2ensembls.nls <- readRDS(paste(OUT_DIR, "rxn2ensembls_nls.Rds", sep = ""))
pca_var_explained <- data.frame(PC = integer(), PVE = numeric(), Reaction = character())

for (rxn_id in names(rxn_pca_list)) {
  rxn_pca <- prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = TRUE)
  summary_pca <- summary(rxn_pca)
  
  n_pcs <- min(10, ncol(summary_pca$importance))
  
  pve <- summary_pca$importance[2, 1:n_pcs]  
  
  pca_var_explained <- rbind(
    pca_var_explained,
    data.frame(PC = 1:n_pcs, PVE = pve, Reaction = rxn_id)
  )
}


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] 9844


In [ ]:
%%R
p <- ggplot(pca_var_explained, aes(x = factor(PC), y = PVE)) +
  geom_violin(trim = FALSE) +                     
  geom_boxplot(width = 0.1, outlier.shape = NA) + 
  labs(x = "Reaction Principal Component", y = "Proportion of Variance Explained") +
  theme_minimal(base_size = 14) +                
  theme(
    panel.border = element_rect(color = "black", fill = NA, size = 1), 
    axis.text = element_text(size = 14),        
    axis.title = element_text(size = 14)       
  )

ggsave("Top10_PCA_violin_boxplot.pdf", plot = p, width = 8, height = 6, dpi = 300)

In [ ]:
%%R
library(scales)
violin_plot <- ggplot(pca_var_explained, aes(x = factor(PC), y = PVE)) +
  geom_violin(trim = FALSE) +              
  geom_boxplot(width = 0.1, outlier.shape = NA) +  
  labs(x = "Reaction Principal Component", y = "Proportion of Variance Explained") +
  scale_x_discrete(labels = paste0("PC", 1:10)) +  
  scale_y_continuous(labels = percent, breaks = seq(0, 1, by = 0.1)) + 
  theme_minimal()

In [8]:
%%R
ggsave(filename = paste(OUT_DIR, "top10_PCA_violin_plot.pdf", sep = ""),
       plot = violin_plot,
       device = "pdf",
       width = 10, height = 6, units = "in", dpi = 300)